In [28]:
from scipy.stats import wilcoxon
import pandas as pd

# Cargar datos
df = pd.read_csv("resultados10.csv")
df.columns = df.columns.str.strip()

col_problema = "PROBLEMAS/METODOS"
referencia = "Limcor"

# Eliminar valores faltantes
df = df.dropna()

df = df.set_index("PROBLEMAS/METODOS")




In [ ]:
df_numerico = df.apply(pd.to_numeric, errors='coerce')

# Calcular estadísticas descriptivas
estadisticas = pd.DataFrame({
    "Media": df_numerico.mean(axis=1),
    "Mediana": df_numerico.median(axis=1),
    "Desviación estándar": df_numerico.std(axis=1),
    "Rango intercuartílico (IQR)": df_numerico.quantile(0.75, axis=1) - df_numerico.quantile(0.25, axis=1),
    "Rango": df_numerico.max(axis=1) - df_numerico.min(axis=1)
})


pd.set_option("display.float_format", "{:.6f}".format)

print("Estadísticas descriptivas:\n")
print(estadisticas)

In [ ]:
# Convertir índice a columna
df = df.reset_index()

# Detectar el nombre real de la columna que antes era índice
print(df.columns.tolist())  # para confirmar
col_problema = df.columns[0]  # primera columna

# Ahora puedes seguir con tu código
problemas = df[col_problema].unique()
resultados = []

print(f"--- Prueba de Wilcoxon por problema ---\n")
for problema in problemas:
    subset = df[df[col_problema] == problema]
    print(f"\n=== Problema: {problema} ===")
    
    for metodo in df.columns:
        if metodo not in [col_problema, referencia]:
            x = subset[referencia]
            y = subset[metodo]
            
            diferencia = x - y
            valores_unicos = diferencia.unique()
            
            if (valores_unicos == 0).all():
                stat, p = 0, 1.0
                significancia = "No"
                print(f"{referencia} vs {metodo} → Estadístico: {stat:.4f}, p-valor: {p:.4e}, Resultados idénticos")
            else:
                stat, p = wilcoxon(x, y, zero_method='pratt')
                significancia = "Sí" if p < 0.05 else "No"
                print(f"{referencia} vs {metodo} → Estadístico: {stat:.4f}, p-valor: {p:.4e}, Significativa: {significancia}")
            
            resultados.append({
                "Problema": problema,
                "Referencia": referencia,
                "Método": metodo,
                "Estadístico": stat,
                "p-valor": round(p, 4),
                "Significativa": significancia
            })

# Guardar resultados en CSV
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("wilcoxon_por_problema.csv", index=False)
